In [ ]:
# # Uncomment this block to install dependencies

# # Install latest anvil
# import sys
# !{sys.executable} -m pip uninstall  -y pyanvil
# !{sys.executable} -m pip install   pyAnVIL==0.0.9rc2 --upgrade
# !{sys.executable} -m pip show pyanvil
# >>> Version: 0.0.9rc2

# # fhirclient needs to be installed separately. see  https://github.com/smart-on-fhir/client-py/issues/70
# !{sys.executable} -m pip install  git+https://github.com/smart-on-fhir/client-py#egg=fhirclient
# !{sys.executable} -m pip show fhirclient


In [1]:
# # Uncomment this block to turn on logging

# import sys
# import logging
# root = logging.getLogger()
# root.setLevel(logging.DEBUG)

# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
# root.addHandler(handler)


In [61]:
def service_account():
    from subprocess import Popen, PIPE
    cmd = ['gcloud', 'config', 'get-value', 'account']
    p = Popen(cmd, stdout=PIPE, stderr=PIPE)
    account, stderr = p.communicate()
    assert p.returncode == 0, "Unable to retrieve service account {stderr}"
    return account.decode("utf-8").rstrip()

# pyAnVIL support for Google's FHIR server

We incorporated `fhirclient`, a flexible Python client for FHIR servers supporting the SMART on FHIR protocol. 

For more information on usage see [smart-on-fhir/client-py](https://github.com/smart-on-fhir/client-py)

In [62]:
# get service account
print(f"logged in as {service_account()}")


logged in as pet-110793006573203727769@terra-test-bwalsh.iam.gserviceaccount.com


## The user should exist in the project and in the healthcare API. Example:

![image](https://user-images.githubusercontent.com/47808/129414919-38136ec6-5e79-41f9-9b42-71aac73d7cf2.png)


In [58]:
# Connect to the server
from anvil.fhir.client import FHIRClient
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://healthcare.googleapis.com/v1/projects/gcp-testing-308520/locations/us-east4/datasets/testset/fhirStores/fhirstore/fhir'
}
smart = FHIRClient(settings=settings)
assert smart.ready, "server should be ready"
print("Server connection ready.")

Server connection ready.


In [59]:
# search for all instances of ResearchStudy
import fhirclient.models.researchstudy as rs
titles = [s.title for s in rs.ResearchStudy.where(struct={}).perform_resources(smart.server)]
assert len(titles) > 0, "Should have at least one ResearchStudy"
print(f"Research Studies: {titles}")

Research Studies: ['1000g-high-coverage-2019', 'my NCPI research study example']
